In [5]:

import string
import pandas as pd 
from tqdm.autonotebook import tqdm

from utils.RagPipeline import Ragpipeline

from utils.utils import normalize_path

In [6]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("../open/sample_submission.csv")

df = pd.read_csv('../open/test.csv')
base_directory = '../open' # Your Base Directory

df.head()

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?


In [7]:
source_list = list(df['Source'].unique())
# 알파벳 리스트
alphabet = list(string.ascii_uppercase)

# 딕셔너리로 파일명과 알파벳 매핑
file_mapping = {alphabet[i]: source_list[i] for i in range(len(source_list))}

reversed_file_mapping = {value: key for key, value in file_mapping.items()}

file_mapping

{'A': '중소벤처기업부_혁신창업사업화자금(융자)',
 'B': '보건복지부_부모급여(영아수당) 지원',
 'C': '보건복지부_노인장기요양보험 사업운영',
 'D': '산업통상자원부_에너지바우처',
 'E': '국토교통부_행복주택출자',
 'F': '「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》',
 'G': '「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》',
 'H': '「FIS 이슈&포커스」 22-2호 《재정성과관리제도》',
 'I': '「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》'}

In [8]:
for i, source in enumerate(source_list):
    
    db_path = f'data/{reversed_file_mapping[source]}'
    pipeline = Ragpipeline(db_path)
    
    target_df = df[df['Source'] == source]
    
    for _, row in tqdm(target_df.iterrows(), total=len(target_df), desc=f"Answering Questions ({i+1}/{len(source_list)})"):
        # 소스 문자열 정규화
        source = normalize_path(row['Source'])
        # 질문 
        question = row['Question']
        
        # 대답
        full_response = pipeline.answer_generation(question)
        first_line_response = full_response.split("\n")[0]
        
        # 결과 저장
        submit_df.loc[submit_df['SAMPLE_ID'] == row['SAMPLE_ID'], 'Answer'] = first_line_response
        

c:\Users\kim_h\anaconda3\envs\final\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Answering Questions (1/9):  11%|█         | 1/9 [00:14<01:55, 14.47s/it]

In [ ]:
submit_df

In [ ]:
submit_df.to_csv("./Faiss_lamma31_ensemble_submission.csv", encoding='UTF-8-sig', index=False)